In [1]:
import json

spell_id_lists = {
        '0' : "Melee",
        '23881' : "Bloodthirst",
        '1680' : "Whirlwind",
        '47475' : "Slam",
        '47450' : "Heroic Strike",
        '47471' : "Execute",
        '12867': "Deep Wounds",
        '12292' : "Death Wish",
        '1719' : "Recklessness",
        '64382' : "Shattering Throw",
        '54758' : "Engi Gloves",
        '2457' : "Battle Stance",
        '2458' : "Berserker Stance",
        '2687' : 'Bloodrage',
        '2825' : 'Bloodlust',
        '47465': 'Rend',
        '7384': 'Overpower',
        '44949': 'Whirlwind OH',
}

# Load Fury Raid Sim Request JSON

In [2]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [3]:
from fury import *

def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))


# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']
reset()

# Default sim agent
Default sim agent is the hardcoded agent from the sim

In [4]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 11891.145775806754


In [5]:
settings['simOptions']['interactive'] = False

reset()
while not wowsims.step():
    pass
    
    
cast_metrics = wowsims.getSpellMetrics()

for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 1, 1, 2, 9, 77]
Death Wish: [2]
Deep Wounds: [133]
Whirlwind: [16]
Recklessness: [1]
Bloodthirst: [33]
Battle Stance: [9]
Berserker Stance: [9]
Bloodrage: [4]
Bloodlust: [1]
Whirlwind OH: [16]
Heroic Strike: [68]
Rend: [4]
Execute: [8]
Slam: [19]
Engi Gloves: [3]
Shattering Throw: [1]
Overpower: [5]


# PPO Agent

## Load Trained PPO Model

In [37]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make
from gymnasium.envs.registration import register

def env_creator(env_config):
    return FurySimEnv(...)

register_env("FurySimEnv", env_creator)

config = PPOConfig()\
            .rollouts(num_rollout_workers=1)

algorithm = config.build(env="FurySimEnv")
algorithm.restore("models\dps-reward_punish-error\PPO\PPO_FurySimEnv_d5a3b_00000_0_2023-05-02_15-25-02\checkpoint_002266")

register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")

2023-05-02 22:02:55,345	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2023-05-02 22:02:55,368	INFO trainable.py:913 -- Restored on 127.0.0.1 from checkpoint: models\dps-reward_punish-error\PPO\PPO_FurySimEnv_d5a3b_00000_0_2023-05-02_15-25-02\checkpoint_002266
2023-05-02 22:02:55,368	INFO trainable.py:922 -- Current state after restoring: {'_iteration': 2266, '_timesteps_total': None, '_time_total': 17449.62928199768, '_episodes_total': 50009}


In [9]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make
from gymnasium.envs.registration import register

register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")
print(env.reset())

(array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,         nan,         nan,
               nan,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.34486885, -0.03487042]), {})


c:\ProgramData\Anaconda3\envs\civ6_rl_rllib\lib\site-packages\gymnasium\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


## Run PPO Model

In [38]:
for i in range(100):
    observation, info = env.reset()
    terminated = False
    reward = 0
    batch = []
    dps_results = np.array([])
    metrics_batch = []
    debug_logs = []
    debug_logs_batch = []
    while not terminated:
        action = algorithm.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        batch.append(np.concatenate((observation, [action], [reward])))
        debug_logs.append(info['debug log'])
    debug_logs_batch.append(debug_logs)
    dps_results = np.append(dps_results, info['dps'])
    metrics_batch.append(info['spell metrics'])

(RolloutWorker pid=24824) 2023-05-02 22:02:55,308	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.


In [ ]:
median = np.argsort(dps_results)[len(dps_results)//2]
print(f'Average DPS: {dps_results[median]}')

cast_metrics = metrics_batch[median]
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

### Debug Log

In [25]:
actions_id = {
    0: "Bloodthirst", 
    1: "Whirlwind",
    2: "Slam",
    3: "HeroicStrike",
    4: "Execute",
    5: "DeathWish",
    6: "Recklessness",
    7: "ShatteringThrow",
    8: "Bloodrage",
    9: "EngiGlove",
    10: "Bloodlust",
    11: "Idle"
}

In [26]:
import pandas as pd

df = pd.DataFrame(debug_logs_batch[median], columns=['Timestamp', 'Action', 'Successful Cast', 'Damage Done', 'Total Damage Done', 'Rage'])
df['Action'] = df['Action'].map(actions_id)
df['DPS'] = df['Total Damage Done'].div(df['Timestamp'])
df

,Timestamp,Action,Successful Cast,Damage Done,Total Damage Done,Rage,DPS
0,0.000000,EngiGlove,True,1919.809658,1919.809658,22.181946,inf
1,0.000000,Bloodlust,True,0.000000,1919.809658,22.181946,inf
2,0.648574,DeathWish,True,0.000000,1919.809658,12.181946,2.960046e+03
3,0.648574,Recklessness,True,0.000000,1919.809658,12.181946,2.960046e+03
4,0.648574,ShatteringThrow,True,0.000000,1919.809658,12.181946,2.960046e+03
...,...,...,...,...,...,...,...
241,59.436223,Bloodrage,False,2514.524495,619162.355334,80.000000,1.041726e+04
242,59.926223,ShatteringThrow,False,0.000000,619162.355334,80.000000,1.033208e+04
243,60.000000,Recklessness,False,0.000000,619162.355334,100.000000,1.031937e+04
244,60.037906,Execute,True,14681.951599,633844.306933,70.000000,1.055740e+04


In [31]:
df.to_csv("debug_logs/debug_log_1.csv")

# Spell Debug

In [ ]:
reset()
Spells.register()
Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])
spell_metrics = []

for i in range(1):
    reset()
    while not wowsims.step():
        print(AutoAttacks.get_swing_time())
        if wowsims.needsInput():
            pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)
    spell_metrics.append(wowsims.getSpellMetrics())

median = np.argsort(damages)[len(damages)//2]
print(f'Average DPS: {damages[median] / duration}')

In [7]:
settings['simOptions']['interactive'] = True

reset()
while not wowsims.step():
    pass
cast_metrics = spell_metrics[median]
for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 54]
Deep Wounds: [67]
Heroic Strike: [54]
